In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM
import os

loader = UnstructuredWordDocumentLoader('./backend/ScrumZero Paper.docx')
docs = loader.load()
docs

[Document(metadata={'source': './backend/ScrumZero Paper.docx'}, page_content="ScrumZero: AI Powered Agile \n\nWorkflow Optimizer\n\nAbstract— In this paper we present Scrum-Zero, an AI-based web application for automating the role of the Scrum Master for Agile teams. Built using FastAPI for the backend, React-like JavaScript frontend, with Google’s Gemini-2.0 generative AI model, Scrum-Zero combines technology into a system that allows daily stand-ups, sprint discipline, and real time to-do prioritization and bottleneck resolution. The system persists tasks across meetings in SQLite and exposes RESTful calls for each phase of the process of task life-cycles. An AI summary module groups team task updates into Completed, In-Progress, and To-Do, and generates an organized and actionable stand-up report. We measure the summary latency of Scrum-Zero, measure it's accuracy against human-sourced reports, and understand user satisfaction, using a controlled study of 30 practitioners. Results 

In [3]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.split_documents(docs)
docs

[Document(metadata={'source': './backend/ScrumZero Paper.docx'}, page_content='ScrumZero: AI Powered Agile \n\nWorkflow Optimizer'),
 Document(metadata={'source': './backend/ScrumZero Paper.docx'}, page_content='Abstract— In this paper we present Scrum-Zero, an AI-based web application for automating the role of the Scrum Master for Agile teams. Built using FastAPI for the backend, React-like JavaScript frontend, with Google’s Gemini-2.0 generative AI model, Scrum-Zero combines technology into a system that allows daily stand-ups, sprint discipline, and real time to-do prioritization and bottleneck resolution. The system persists tasks across meetings in SQLite and exposes RESTful calls for each phase'),
 Document(metadata={'source': './backend/ScrumZero Paper.docx'}, page_content="The system persists tasks across meetings in SQLite and exposes RESTful calls for each phase of the process of task life-cycles. An AI summary module groups team task updates into Completed, In-Progress, and

In [4]:
embeddings = OllamaEmbeddings(model='nomic-embed-text')
base_dir = "./backend/bases/chromadb"
vectordb = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=base_dir
)
vectordb


C:\Users\USER\AppData\Local\Temp\ipykernel_13152\840076577.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model='nomic-embed-text')


In [ ]:
retriever = vectordb.as_retriever()
retriever.invoke('What is ScrumZero?')

[Document(id='11d00099-10b3-4cb5-999b-7fe6fce7f7fd', metadata={'source': './backend/ScrumZero Paper.docx'}, page_content='ScrumZero: AI Powered Agile \n\nWorkflow Optimizer'),
 Document(id='4b9f8614-f58d-4f56-8fcc-d04ad8771d67', metadata={'source': './backend/ScrumZero Paper.docx'}, page_content='time spent managing manual oversight. By removing repetitive decisions, communication bottlenecks and providing automation of agile routines, Scrum AI not only provides improved efficiencies, but it also allows teams to focus on development and innovation in total. The results provide evidence that AI-powered Scrum facilitation is both technically and practically\xa0viable.'),
 Document(id='6fca76b4-caf0-4693-95f9-1502a070eaf0', metadata={'source': './backend/ScrumZero Paper.docx'}, page_content='This work makes the following contributions:\n\nThe design and implementation of an AI-powered Scrum Master replacement system for Agile software teams.\n\nThe use of cutting-edge generative AI to gen

In [9]:
llm = OllamaLLM(model='gemma:2b')
qa_retriever = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type='refine')
qa_retriever

RetrievalQA(verbose=False, combine_documents_chain=RefineDocumentsChain(verbose=False, initial_llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context_str', 'question'], input_types={}, partial_variables={}, template='Context information is below. \n------------\n{context_str}\n------------\nGiven the context information and not prior knowledge, answer the question: {question}\n'), llm=OllamaLLM(model='gemma:2b'), output_parser=StrOutputParser(), llm_kwargs={}), refine_llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context_str', 'existing_answer', 'question'], input_types={}, partial_variables={}, template="The original question is as follows: {question}\nWe have provided an existing answer: {existing_answer}\nWe have the opportunity to refine the existing answer (only if needed) with some more context below.\n------------\n{context_str}\n------------\nGiven the new context, refine the original answer to better answer the question. I

In [10]:
query='What is ScrumZero'
answer = qa_retriever.run(query)
answer

C:\Users\USER\AppData\Local\Temp\ipykernel_13152\1556719636.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_retriever.run(query)


"## Refined Answer:\n\nScrumZero is an AI tool that assists software development teams by automating repetitive and time-consuming manual tasks within the agile development process. By leveraging advanced machine learning algorithms, ScrumZero optimizes team workflows and facilitates smoother communication. This results in accelerated delivery of value and improved overall development efficiency.\n\nScrumZero offers several functionalities that enhance the agile development process:\n\n* **Automates manual tasks:** It eliminates the need for manual testers to perform repetitive testing, freeing up their time for higher-value activities.\n* **Real-time insights:** Utilizes machine learning to analyze data and provide real-time insights and recommendations, guiding the development team's efforts.\n* **Streamlines communication:** Automates communication between team members, reducing delays and bottlenecks in the workflow.\n* **Improves workflow:** By streamlining manual tasks and optimi